In [1]:
from all_imports import *
import requests
import csv
import _modules 

 ### test init### 


In [4]:
# Configuration
API_KEY = _modules.SCOPUS_API_KEY # Replace with your Scopus API key
SEARCH_QUERY = 'TITLE-ABS-KEY("outdoor thermal comfort") AND PUBYEAR AFT 1999 AND PUBYEAR BEF 2025'
BASE_URL = 'https://api.elsevier.com/content/search/scopus'
CSV_FILENAME = 'scopus_outdoor_thermal_comfort.csv'

# Function to fetch papers from Scopus API
def fetch_papers(api_key, query):
    headers = {
        'X-ELS-APIKey': api_key,
        'Accept': 'application/json'
    }
    
    params = {
        'query': query,
        'view': 'STANDARD',  # You can change this to 'COMPLETE' if your access allows
        'count': 25  # Number of results per page
    }

    papers = []
    start_index = 0
    
    while True:
        params['start'] = start_index
        response = requests.get(BASE_URL, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Error fetching data: {response.status_code}, {response.text}")
            break

        data = response.json()

        if 'search-results' not in data or 'entry' not in data['search-results']:
            break

        entries = data['search-results']['entry']
        if not entries:
            break

        for entry in entries:
            doi = entry.get('prism:doi', 'N/A')
            title = entry.get('dc:title', 'N/A')
            authors = entry.get('dc:creator', 'N/A')
            publication_date = entry.get('prism:coverDate', 'N/A')
            volume = entry.get('prism:volume', 'N/A')
            issue = entry.get('prism:issueIdentifier', 'N/A')
            keywords = entry.get('authkeywords', 'N/A')
            document_type = entry.get('subtypeDescription', 'N/A')

            if isinstance(keywords, list):
                keywords = '; '.join(keywords)

            papers.append({
                'DOI': doi,
                'Title': title,
                'Authors': authors,
                'Publication Date': publication_date,
                'Volume': volume,
                'Issue': issue,
                'Keywords': keywords,
                'Document Type': document_type
            })

        start_index += len(entries)
        
    return papers

# Function to save papers to CSV
def save_to_csv(filename, papers):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['DOI', 'Title', 'Authors', 'Publication Date', 'Volume', 'Issue', 'Keywords', 'Document Type'])
        writer.writeheader()
        for paper in papers:
            writer.writerow(paper)
    print(f"Data saved to {filename}")

# Main script
if __name__ == "__main__":
    papers = fetch_papers(API_KEY, SEARCH_QUERY)
    save_to_csv(CSV_FILENAME, papers)

Data saved to scopus_outdoor_thermal_comfort.csv
